In [1]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
list(iris)

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [2]:
iris.data.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [3]:
iris.target_names


array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
from sklearn.model_selection import train_test_split

X = iris.data[:100]
y = iris.target[:100]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
import matplotlib.pyplot as plt
import numpy as np


Linear regression

In [6]:
X_lr = iris.data[:100]
y_lr = iris.target[:100]
X_lr_train, X_lr_test, y_lr_train, y_lr_test = train_test_split(X, y, random_state=42)

print("Test values", X_lr_test.iloc[10].tolist(), y_lr_test[10])

X_lr_train = np.column_stack([np.ones(len(X_lr_train)), np.array(X_lr_train)]) 
y_lr_train = np.array(y_lr_train)

theta_lr = np.random.random(5)


theta_lr = np.linalg.inv(X_lr_train.T @ X_lr_train) @ X_lr_train.T @ y_lr_train
print(theta_lr)
pred_y_lr = theta_lr.T @ np.array([1.0, 5.7, 3.8, 1.7, 0.3])

print(pred_y_lr)

Test values [5.7, 3.8, 1.7, 0.3] 0
[ 0.38052393 -0.03729905 -0.16140776  0.20334807  0.31790473]
-0.004366993508547787


In [7]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_lr_train, y_lr_train)
print(lin_reg.intercept_, lin_reg.coef_)

lin_reg.predict(np.array([1, 5.7, 3.8, 1.7, 0.3]).reshape(1, -1))

0.3805239261489818 [ 0.         -0.03729905 -0.16140776  0.20334807  0.31790473]


array([-0.00436699])

In [8]:
X_test.iloc[2].tolist(), y_test.iloc[2]

([5.9, 3.2, 4.8, 1.8], 1)

In [9]:
y_train.unique()

array([0, 1])

In [10]:
import pandas as pd

X_gd = iris.data
y_gd = iris.target
a = np.array(y_gd)
y_gd = np.zeros((a.size, a.max() + 1))
y_gd[np.arange(a.size), a] = 1


In [11]:
X_gd_train, X_gd_test, y_gd_train, y_gd_test = train_test_split(X_gd, y_gd, random_state=42)

In [103]:
def softmax_t(logits):
    exps = np.exp(logits)
    exp_sums = exps.sum(axis=1, keepdims=True)
    return exps / exp_sums
    
def sigmod(X):
  return 1/(1+np.exp(X))

In [104]:
eta = 0.005  # learning rate
n_epochs = 5000
X_b = np.column_stack([np.ones(len(X_gd_train)), X_gd_train])
m = len(X_b)  # number of instances
epsilon = 1e-5

np.random.seed(42)
theta = np.random.randn(5, 3)  # randomly initialized model parameters

rmse_val = []
rmse_train = []

for epoch in range(n_epochs):

  logits = X_b @ theta 
  Y_proba = softmax_t(logits)

  if epoch % 100 == 0:
    Y_proba_valid = softmax_t(np.column_stack([np.ones(len(X_gd_test)) ,X_gd_test]) @ theta)
    xentropy_losses = -(y_gd_test * np.log(Y_proba_valid + epsilon))
    print(epoch, xentropy_losses.sum(axis=1).mean())

  error = Y_proba - y_gd_train
  gradients = 1 / m * X_b.T @ error
  theta = theta - eta * gradients

print(theta)


0 6.968296275250234
100 0.9506370866185793
200 0.7515963347470777
300 0.643302408477065
400 0.5779572117510174
500 0.5340410772771417
600 0.501948717038097
700 0.4769891967333524
800 0.45665930462099535
900 0.4395183199446676
1000 0.4246833766152724
1100 0.4115845455312387
1200 0.3998374809448111
1300 0.3891731775492506
1400 0.37939722995270436
1500 0.37036516374006856
1600 0.36196693319275086
1700 0.3541168602827945
1800 0.34674691823029474
1900 0.3398021348358257
2000 0.3332373762017882
2100 0.32701505124537966
2200 0.32110344376944316
2300 0.3154754805762991
2400 0.3101078078857917
2500 0.30498008922396147
2600 0.30007446473045785
2700 0.2953751296986371
2800 0.29086800228538034
2900 0.2865404586830989
3000 0.28238111988793374
3100 0.2783796783361322
3200 0.2745267556475459
3300 0.27081378486653596
3400 0.26723291216689726
3500 0.2637769141537216
3600 0.2604391277660163
3700 0.2572133904398312
3800 0.2540939886897475
3900 0.25107561364777814
4000 0.24815332239263652
4100 0.245322504

In [111]:
logits = np.column_stack([np.ones(len(X)) ,X]) @ theta
Y_proba = softmax_t(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y).mean()
accuracy_score

0.96